# Number of Discordant Pairs Country

Discordant pair is a pair contain of male-female and its prediction, such that the Sentiment Analysis produce a different prediction. 
Example of discordant pair from country: 

`<(name from Indonesia, prediction), (name from USA, prediction)>`

`<(“Paijo is angry”, 1), (“John is angry”, 0)>`

In [78]:
import pandas as pd
import numpy as np
import math
import time

In [79]:
eval_dir = "occupation"
# df = pd.read_csv("../data/" + eval_dir + "/test.csv", header=None, sep="\t", names=[0, 1, "mutant", "template", "gender", "label", "country"])
df = pd.read_csv("../data/" + eval_dir + "/test.csv", header=None, sep="\t", names=[0, 1, "mutant", "template", "occupation"])
df

,0,1,mutant,template,occupation
0,0,Why do movie makers always go against the atte...,Why do movie makers always go against the atte...,Why do movie makers always go against @DetOTHE...,attendant
1,0,Why do movie makers always go against the cash...,Why do movie makers always go against the cash...,Why do movie makers always go against @DetOTHE...,cashier
2,0,Why do movie makers always go against the teac...,Why do movie makers always go against the teac...,Why do movie makers always go against @DetOTHE...,teacher
3,0,Why do movie makers always go against the nurs...,Why do movie makers always go against the nurs...,Why do movie makers always go against @DetOTHE...,nurse
4,0,Why do movie makers always go against the assi...,Why do movie makers always go against the assi...,Why do movie makers always go against @DetOTHE...,assistant
...,...,...,...,...,...
39495,1,This 1953 Sam Fuller movie contains some of hi...,This 1953 Sam Fuller movie contains some of hi...,This 1953 Sam Fuller movie contains some of hi...,nutritionist
39496,1,This 1953 Sam Fuller movie contains some of hi...,This 1953 Sam Fuller movie contains some of hi...,This 1953 Sam Fuller movie contains some of hi...,photographer
39497,1,This 1953 Sam Fuller movie contains some of hi...,This 1953 Sam Fuller movie contains some of hi...,This 1953 Sam Fuller movie contains some of hi...,police officer
39498,1,This 1953 Sam Fuller movie contains some of hi...,This 1953 Sam Fuller movie contains some of hi...,This 1953 Sam Fuller movie contains some of hi...,singer


In [80]:
def read_txt(fpath):
    pred = []
    file = open(fpath)
    lines = file.readlines()
    for l in lines :
        pred.append(int(l))
    file.close()
    
    return pred

In [81]:
output_dir = "occupation"

result_dir = "../result/" + output_dir + "/"

path = result_dir + "results_data.txt"

pred = read_txt(path)

print(len(pred))

39500


In [82]:
df["prediction"] = pred

In [83]:
df = df.drop(columns=[0,1])
df.head()

,mutant,template,occupation,prediction
0,Why do movie makers always go against the atte...,Why do movie makers always go against @DetOTHE...,attendant,0
1,Why do movie makers always go against the cash...,Why do movie makers always go against @DetOTHE...,cashier,0
2,Why do movie makers always go against the teac...,Why do movie makers always go against @DetOTHE...,teacher,0
3,Why do movie makers always go against the nurs...,Why do movie makers always go against @DetOTHE...,nurse,0
4,Why do movie makers always go against the assi...,Why do movie makers always go against @DetOTHE...,assistant,0


### Use Groupby to Group the text by Template

In [84]:
df["template"] = df["template"].astype("category")
df["template_id"] = df["template"].cat.codes

In [85]:
gb = df.groupby("template_id")

In [86]:
gb.count()

,mutant,template,occupation,prediction
template_id,,,,
0,79,79,79,79
1,79,79,79,79
2,79,79,79,79
3,79,79,79,79
4,79,79,79,79
...,...,...,...,...
495,79,79,79,79
496,79,79,79,79
497,79,79,79,79


In [87]:
len(gb.size())

500

#### Create Discordant Pairs based on Country

You can create another discordant pairs from another column by changing the `identifier`.

In case you want to calculate discordant pairs from gender, you can try `identifier = "gender"`

In [88]:
start = time.time()

identifier = "occupation"

mutant_text_1 = []
mutant_text_2 = []
prediction_1 = []
prediction_2 = []
identifier_1 = []
identifier_2 = []
template = []
for i in range(len(gb.size())) :
    data = gb.get_group(i)
    dc = data.groupby(identifier)
    for k1, v1 in dict(iter(dc)).items() :
        for k2, v2 in dict(iter(dc)).items() :
            if k1 != k2 :
                for m_1, p_1, i_1, t in zip(v1["mutant"].values, v1["prediction"].values, v1[identifier].values, v1["template"].values) :
                    for m_2, p_2, i_2 in zip(v2["mutant"].values, v2["prediction"].values, v2[identifier].values) :
                        mutant_text_1.append(m_1)
                        prediction_1.append(p_1)
                        identifier_1.append(i_1)
                        mutant_text_2.append(m_2)
                        prediction_2.append(p_2)
                        identifier_2.append(i_2)
                        template.append(t)

end = time.time()
print("Execution time: ", end-start)

Execution time:  1106.6335020065308


In [99]:
df = pd.DataFrame(data={"mutant_1" : mutant_text_1, "mutant_2" : mutant_text_2, "prediction_1": prediction_1, "prediction_2" : prediction_2, "identifier_1": identifier_1, "identifier_2" : identifier_2, "template": template})

df

,mutant_1,mutant_2,prediction_1,prediction_2,identifier_1,identifier_2,template
0,""" A wrong-doer is often a man that has left so...",""" A wrong-doer is often a man that has left so...",1,1,CEO,accountant,""" A wrong-doer is often a man that has left so..."
1,""" A wrong-doer is often a man that has left so...",""" A wrong-doer is often a man that has left so...",1,1,CEO,air traffic controller,""" A wrong-doer is often a man that has left so..."
2,""" A wrong-doer is often a man that has left so...",""" A wrong-doer is often a man that has left so...",1,1,CEO,analyst,""" A wrong-doer is often a man that has left so..."
3,""" A wrong-doer is often a man that has left so...",""" A wrong-doer is often a man that has left so...",1,1,CEO,architect,""" A wrong-doer is often a man that has left so..."
4,""" A wrong-doer is often a man that has left so...",""" A wrong-doer is often a man that has left so...",1,1,CEO,artist,""" A wrong-doer is often a man that has left so..."
...,...,...,...,...,...,...,...
3080995,"waste of my life, .... the writer should be em...","waste of my life, .... the teacher should be e...",0,0,writer,teacher,"waste of my life, .... @DetOTHER @OCCUPATION s..."
3080996,"waste of my life, .... the writer should be em...","waste of my life, .... the therapist should be...",0,0,writer,therapist,"waste of my life, .... @DetOTHER @OCCUPATION s..."
3080997,"waste of my life, .... the writer should be em...","waste of my life, .... the undertaker should b...",0,0,writer,undertaker,"waste of my life, .... @DetOTHER @OCCUPATION s..."
3080998,"waste of my life, .... the writer should be em...","waste of my life, .... the veterinarian should...",0,0,writer,veterinarian,"waste of my life, .... @DetOTHER @OCCUPATION s..."


In [100]:
df["discordant"] = df["prediction_1"] != df["prediction_2"]
df

,mutant_1,mutant_2,prediction_1,prediction_2,identifier_1,identifier_2,template,discordant
0,""" A wrong-doer is often a man that has left so...",""" A wrong-doer is often a man that has left so...",1,1,CEO,accountant,""" A wrong-doer is often a man that has left so...",False
1,""" A wrong-doer is often a man that has left so...",""" A wrong-doer is often a man that has left so...",1,1,CEO,air traffic controller,""" A wrong-doer is often a man that has left so...",False
2,""" A wrong-doer is often a man that has left so...",""" A wrong-doer is often a man that has left so...",1,1,CEO,analyst,""" A wrong-doer is often a man that has left so...",False
3,""" A wrong-doer is often a man that has left so...",""" A wrong-doer is often a man that has left so...",1,1,CEO,architect,""" A wrong-doer is often a man that has left so...",False
4,""" A wrong-doer is often a man that has left so...",""" A wrong-doer is often a man that has left so...",1,1,CEO,artist,""" A wrong-doer is often a man that has left so...",False
...,...,...,...,...,...,...,...,...
3080995,"waste of my life, .... the writer should be em...","waste of my life, .... the teacher should be e...",0,0,writer,teacher,"waste of my life, .... @DetOTHER @OCCUPATION s...",False
3080996,"waste of my life, .... the writer should be em...","waste of my life, .... the therapist should be...",0,0,writer,therapist,"waste of my life, .... @DetOTHER @OCCUPATION s...",False
3080997,"waste of my life, .... the writer should be em...","waste of my life, .... the undertaker should b...",0,0,writer,undertaker,"waste of my life, .... @DetOTHER @OCCUPATION s...",False
3080998,"waste of my life, .... the writer should be em...","waste of my life, .... the veterinarian should...",0,0,writer,veterinarian,"waste of my life, .... @DetOTHER @OCCUPATION s...",False


In [91]:
print("Number of Discordant Pairs: ", len(df[df["discordant"] == True]))

Number of Discordant Pairs:  896


In [94]:
print("Discordant Pairs Rate: ", len(df[df["discordant"] == True]) / len(df) )

Discordant Pairs Rate:  0.000290814670561506


In [95]:
d = df[df["discordant"] == True]
d = d.drop(columns=["discordant"])

In [98]:
for id, rows in d.iloc[100:101].iterrows():
    print()
    print("Mutant 1:")
    print(rows["mutant_1"])
    print("Mutant 2:")
    print(rows["mutant_2"])
    print("Prediction 1:")
    print(rows["prediction_1"])
    print("Prediction 2:")
    print(rows["prediction_2"])
    print("Identifier 1:")
    print(rows["identifier_1"])
    print("Identifier 2:")
    print(rows["identifier_2"])
    print("Template:")
    print(rows["template"])


Mutant 1:
French Cinema sucks! Down with all these psychiotric visions with their my-God-am-I-cultivated distinguished attitudes! Pestilence to conceited symbolic film-language and impervious chiffres! I'll no longer have a mind for that! Léos Carax, did you ever think about, that a dialogue in a film could be natural and vivid ??? Maybe I'm too common to understand you? Or had it been your task to confirm all the clichés of a Frenchman the world can have? Guillaume the to-be-guilliotined comes to his home-palace, Mme. Deneuve, not in the picture, plays the flute: " Here am I, darling! " In this moment, I knew, that she's in the bathtub, and we `ll see her lying in there soon. Don't misunderstand me, I'm not prudish, and the incestous sex scene was the climax of the film. But this is, in Berlin, we say " etepetete ", what means something like " être-peut-être ", a snobistic, self-satisfied, and, the worst, seen that often in French movies I can tell! Other example: She, beautiful and 